In [1]:
from keras import layers
from keras import models

In [2]:
model = models.Sequential()

'''
    Данная сеть принимает на входе
    тензор с формой (высота_изображения,
    ширина_изображения, каналы).
    
    Изменение высоты и ширины сжимаются
    с ростом глубины сети. Количество каналов
    управляется первым арнументом,
    передаваемым в слои Conc2D (32 / 64).
'''

# сверточная состовляющая
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [3]:
'''
    Теперь необходимо передать после
    выходного тензора с формой (3, 3, 64)
    на вход полносвязной класифицирующей сети.
    
    Выходы (3, 3, 64) преобразуются в векторы
    (576,) перед передачей двум слоям Dense.
'''

# полносвязная сеть
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [4]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [11]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

train_images.shape

(60000, 28, 28, 1)

In [5]:
# Сбор и подготовка данных к обучению и тестировке
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
    
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [12]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 14s 14ms/step - loss: 1.6921 - accuracy: 0.4122
Epoch 2/5
938/938 [==============================] - 13s 14ms/step - loss: 0.2171 - accuracy: 0.9322
Epoch 3/5
938/938 [==============================] - 13s 14ms/step - loss: 0.1054 - accuracy: 0.9682
Epoch 4/5
938/938 [==============================] - 13s 14ms/step - loss: 0.0679 - accuracy: 0.9794
Epoch 5/5
938/938 [==============================] - 13s 14ms/step - loss: 0.0507 - accuracy: 0.9838


In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 1s 2ms/step - loss: 0.0374 - accuracy: 0.9894


0.9894000291824341

In [8]:
# Модель без max-polling
'''
    1. Данная модель не способствует изучению
    пространственной иерархии признаков.
    
    2. Заключительная карта признаков имеет 
    22 * 22 * 64 = 30976 признаков на образец.
'''
model_no_max_poll = models.Sequential()
model_no_max_poll.add(layers.Conv2D(32, (3, 3),
                      activation='relu',
                      input_shape=(28, 28, 1)))
model_no_max_poll.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_no_max_poll.add(layers.Conv2D(64, (3, 3), activation='relu'))

model_no_max_poll.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________
